## CNN sentence classifier - Rerun of training using only the first 21 tokens - the average sentence length.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 21


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 11.2 ms, sys: 11.9 ms, total: 23.1 ms
Wall time: 22.4 ms


## Tokenize the training data set and build a vocabulary from it


In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 31.3 s, sys: 134 ms, total: 31.4 s
Wall time: 31 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:
%%time
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2ballen21.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models  (Removed the filter sizes 32 or 64 that exceed the size of the shorter sequence)
for k_sizes in [ [8,16], [4,8,12], [4,8,16], [8,12,16] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal21",
                      model_dir        = "../models/CNNBal21",
                      max_len          = max_len)
            
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 8ms/step - loss: 0.6599 - accuracy: 0.6293 - val_loss: 0.6750 - val_accuracy: 0.6108
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000214/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4558 - accuracy: 0.7856 - val_loss: 0.3319 - val_accuracy: 0.8580
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000214/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2712 - accuracy: 0.8943 - val_loss: 0.2872 - val_accuracy: 0.8797
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000214/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1641 - accuracy: 0.9417 - val_loss: 0.1977 - val_accuracy: 0.9268
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000214/assets
Epoch 5/20
288/288 [=====

Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0566 - accuracy: 0.9810 - val_loss: 0.2467 - val_accuracy: 0.9250
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000323/assets
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0386 - accuracy: 0.9869 - val_loss: 0.3075 - val_accuracy: 0.9173
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0257 - accuracy: 0.9920 - val_loss: 0.2939 - val_accuracy: 0.9268
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000323/assets
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0189 - accuracy: 0.9944 - val_loss: 0.3727 - val_accuracy: 0.9150
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0207 - accuracy: 0.9935 - val_loss: 0.5617 - val_accuracy: 0.8834
Epoch 11/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0165 - accuracy: 0.9950 - val_loss: 0.3519 - val_accuracy: 0.9

288/288 [==============================] - 1s 5ms/step - loss: 0.0136 - accuracy: 0.9956 - val_loss: 0.3033 - val_accuracy: 0.9439
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000430/assets
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0094 - accuracy: 0.9979 - val_loss: 0.1731 - val_accuracy: 0.9647
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000430/assets
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0090 - accuracy: 0.9978 - val_loss: 0.3180 - val_accuracy: 0.9417
Epoch 15/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0109 - accuracy: 0.9961 - val_loss: 0.3354 - val_accuracy: 0.9471
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0134 - accuracy: 0.9952 - val_loss: 0.2923 - val_accuracy: 0.9485
Epoch 17/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0111 - accuracy: 0.9953 - val_loss: 0.3836 - val_accuracy: 0.9394
Epoc

288/288 [==============================] - 1s 5ms/step - loss: 0.0090 - accuracy: 0.9965 - val_loss: 0.6296 - val_accuracy: 0.9091
Epoch 20/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0057 - accuracy: 0.9984 - val_loss: 0.6386 - val_accuracy: 0.8978



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 6ms/step - loss: 0.6300 - accuracy: 0.6429 - val_loss: 0.6320 - val_accuracy: 0.6673
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000611/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4230 - accuracy: 0.8111 - val_loss: 0.3216 - val_accuracy: 0.8599
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000611/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2444 - accuracy: 0.9059 - va

288/288 [==============================] - 1s 5ms/step - loss: 0.2194 - accuracy: 0.9154 - val_loss: 0.2756 - val_accuracy: 0.8987
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000721/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1089 - accuracy: 0.9620 - val_loss: 0.2726 - val_accuracy: 0.9078
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000721/assets
Epoch 5/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0607 - accuracy: 0.9798 - val_loss: 0.3610 - val_accuracy: 0.8956
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0381 - accuracy: 0.9872 - val_loss: 0.2451 - val_accuracy: 0.9326
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000721/assets
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0244 - accuracy: 0.9916 - val_loss: 0.2207 - val_accuracy: 0.9421
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-000721/assets
E

...

INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-003112/assets
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0124 - accuracy: 0.9969 - val_loss: 0.3139 - val_accuracy: 0.9268
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0105 - accuracy: 0.9978 - val_loss: 0.2098 - val_accuracy: 0.9485
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-003112/assets
Epoch 10/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0090 - accuracy: 0.9975 - val_loss: 0.3326 - val_accuracy: 0.9304
Epoch 11/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0124 - accuracy: 0.9964 - val_loss: 0.3253 - val_accuracy: 0.9367
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0074 - accuracy: 0.9981 - val_loss: 0.3220 - val_accuracy: 0.9349
Epoch 13/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0093 - accuracy: 0.9968 - val_loss: 0.5378 - val_accuracy: 0

288/288 [==============================] - 2s 5ms/step - loss: 0.0073 - accuracy: 0.9978 - val_loss: 0.3848 - val_accuracy: 0.9290
Epoch 14/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0185 - accuracy: 0.9941 - val_loss: 0.3208 - val_accuracy: 0.9448
Epoch 15/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0113 - accuracy: 0.9962 - val_loss: 0.5007 - val_accuracy: 0.9123
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0232 - accuracy: 0.9918 - val_loss: 0.4343 - val_accuracy: 0.9263
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.3322 - val_accuracy: 0.9444
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0090 - accuracy: 0.9974 - val_loss: 0.4462 - val_accuracy: 0.9227
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0070 - accuracy: 0.9978 - val_loss: 0.7701 - val_accuracy: 0.8915
Epoch 20/20




##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 64
Adding Convolution: Kernel Size: 12, Filter Count: 64
Adding Convolution: Kernel Size: 16, Filter Count: 64
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6199 - accuracy: 0.6565 - val_loss: 0.4681 - val_accuracy: 0.7798
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-003417/assets
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.3445 - accuracy: 0.8543 - val_loss: 0.2311 - val_accuracy: 0.9064
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-003417/assets
Epoch 3/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1674 - accuracy: 0.9407 - val_loss: 0.2514 - val_accuracy: 0.8960
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0694 - accuracy: 0.9795 - val_loss: 0.1660 - val_accuracy: 0.9448
INFO:tensorflow:Assets written to: ../mo

288/288 [==============================] - 2s 6ms/step - loss: 0.0189 - accuracy: 0.9938 - val_loss: 0.2216 - val_accuracy: 0.9507
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-003529/assets
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0277 - accuracy: 0.9912 - val_loss: 0.2646 - val_accuracy: 0.9453
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0158 - accuracy: 0.9947 - val_loss: 0.2663 - val_accuracy: 0.9444
Epoch 10/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0130 - accuracy: 0.9964 - val_loss: 0.1883 - val_accuracy: 0.9661
INFO:tensorflow:Assets written to: ../models/CNNBal21/210409-003529/assets
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0231 - accuracy: 0.9919 - val_loss: 0.3832 - val_accuracy: 0.9295
Epoch 12/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0162 - accuracy: 0.9948 - val_loss: 0.5041 - val_accuracy: 0.9001
Epoch 

Epoch 14/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0064 - accuracy: 0.9979 - val_loss: 0.3277 - val_accuracy: 0.9394
Epoch 15/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0212 - accuracy: 0.9914 - val_loss: 0.4305 - val_accuracy: 0.9182
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0084 - accuracy: 0.9966 - val_loss: 0.5657 - val_accuracy: 0.9037
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0046 - accuracy: 0.9984 - val_loss: 0.5300 - val_accuracy: 0.9168
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0049 - accuracy: 0.9985 - val_loss: 0.5277 - val_accuracy: 0.9195
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0091 - accuracy: 0.9973 - val_loss: 0.5241 - val_accuracy: 0.9118
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 8.6462e-04 - accuracy: 1.0000 - val_loss: 0.2849 - val_accuracy: 0.





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/